# Introduction

This is a project I created using guided steps from Dataquest.io. The aim of the project is to practice using Pandas and SQLite to clean and normalize a dataset prior to analysis. The data deals with Academy Awards nominations, and was downloaded in CSV format from https://www.aggdata.com/awards/oscar, along with a list of ceremony host names sourced from Wikipedia. 

# Preparing the dataset for SQLite

The first step is cleaning the data. This includes removing unnecessary columns, separating columns that contain numerous data points into distinct columns, and formatting the data contained in the colums so as to be more useful. 

In [22]:
#Import pandas and read in the dataset. Look at the shape and first few lines of the dataset.

import pandas as pd
df = pd.read_csv("academy_awards.csv", encoding="ISO-8859-1")
print(df.shape)
df.head()

(10137, 11)


,Year,Category,Nominee,Additional Info,Won?,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,2010 (83rd),Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,NaN,NaN,NaN,NaN,NaN,NaN
1,2010 (83rd),Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,NaN,NaN,NaN,NaN,NaN,NaN
2,2010 (83rd),Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},NO,NaN,NaN,NaN,NaN,NaN,NaN
3,2010 (83rd),Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},YES,NaN,NaN,NaN,NaN,NaN,NaN
4,2010 (83rd),Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},NO,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# Find out what's in the "Unnamed" columns.

print(df["Unnamed: 5"].value_counts())
print(df["Unnamed: 6"].value_counts())
print(df["Unnamed: 7"].value_counts())
print(df["Unnamed: 8"].value_counts())
print(df["Unnamed: 9"].value_counts())
print(df["Unnamed: 10"].value_counts())

*                                                                                                               7
 discoverer of stars                                                                                            1
 D.B. "Don" Keele and Mark E. Engebretson has resulted in the over 20-year dominance of constant-directivity    1
 error-prone measurements on sets. [Digital Imaging Technology]"                                                1
 resilience                                                                                                     1
Name: Unnamed: 5, dtype: int64
*                                                                   9
 sympathetic                                                        1
 direct radiator bass style cinema loudspeaker systems. [Sound]"    1
 flexibility and water resistance                                   1
Name: Unnamed: 6, dtype: int64
 kindly                                               1
*                               

Out of over 10,000 rows of data, there are only a few rows that contain data in these columns. The entries in these columns appear to be random notes and will most likely not be helpful for our data analysis. We will filter them out. First, however, we will clean up the "Year" and "Won" columns.

In [24]:
# Remove ordinal year and leave only the 4-digit year.

print(df["Year"].head())
df["Year"] = df["Year"].str[0:4]
df["Year"] = df["Year"].astype(int)
df["Year"].head()

0    2010 (83rd)
1    2010 (83rd)
2    2010 (83rd)
3    2010 (83rd)
4    2010 (83rd)
Name: Year, dtype: object


0    2010
1    2010
2    2010
3    2010
4    2010
Name: Year, dtype: int64

In [25]:
# Create a new column that indicates whether the nominee won using 0 for "no" and 1 for "yes".
# This is important because SQLite uses 0 and 1 to represent Boolean values.

boolean = {"NO": 0, "YES": 1}
df["Won"] = df["Won?"].map(boolean)
df.head()


,Year,Category,Nominee,Additional Info,Won?,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Won
0,2010,Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,2010,Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,2010,Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},NO,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,2010,Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},YES,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,2010,Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},NO,NaN,NaN,NaN,NaN,NaN,NaN,0.0


Now we will create a dataframe containing only the rows of data we want to work with. For the purposes of this project, we will focus on nominations for actors and actresses in leading and supporting roles, starting in the year 2000. 

In [26]:
later_than_2000 = df[df["Year"]>2000]
award_categories = ["Actor -- Leading Role", 
                    "Actor -- Supporting Role", 
                    "Actress -- Leading Role", 
                    "Actress -- Supporting Role"]

nominations = later_than_2000[later_than_2000["Category"].isin(award_categories)]
nominations.head()

,Year,Category,Nominee,Additional Info,Won?,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Won
0,2010,Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,2010,Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,2010,Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},NO,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,2010,Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},YES,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,2010,Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},NO,NaN,NaN,NaN,NaN,NaN,NaN,0.0


Let's see if the "Unnamed" columns contain any information in our new dataframe.

In [27]:
print(nominations["Unnamed: 5"].value_counts())
print(nominations["Unnamed: 6"].value_counts())
print(nominations["Unnamed: 7"].value_counts())
print(nominations["Unnamed: 8"].value_counts())
print(nominations["Unnamed: 9"].value_counts())
print(nominations["Unnamed: 10"].value_counts())

Series([], Name: Unnamed: 5, dtype: int64)
Series([], Name: Unnamed: 6, dtype: int64)
Series([], Name: Unnamed: 7, dtype: int64)
Series([], Name: Unnamed: 8, dtype: int64)
Series([], Name: Unnamed: 9, dtype: int64)
Series([], Name: Unnamed: 10, dtype: int64)


Nope. Let's get rid of them. We also want to get rid of the extra "Won?" column.

In [28]:
to_drop = ["Won?", "Unnamed: 5", "Unnamed: 6", "Unnamed: 7", "Unnamed: 8", "Unnamed: 9", "Unnamed: 10"]
final_nominations = nominations.drop(to_drop, axis=1)
final_nominations["Won"] = final_nominations["Won"].astype(int)
final_nominations.head()

,Year,Category,Nominee,Additional Info,Won
0,2010,Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},0
1,2010,Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},0
2,2010,Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},0
3,2010,Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},1
4,2010,Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},0


This is looking much better. Now let's separate the "Additional Info" column into separate "Movie" and "Character" columns.

In [29]:
# Remove the apostrophe and bracket at the end of the line.
additional_info_one = final_nominations["Additional Info"].str.rstrip("'}")
print(additional_info_one.head())

# Split the string on the remaining bracket and apostrophe, creating a list containing the movie title and character name.
additional_info_two = additional_info_one.str.split(" {'")
print(additional_info_two.head())

# Create the appropriate columns for the resulting data.
final_nominations["Movie"] = additional_info_two.str[0]
final_nominations["Character"] = additional_info_two.str[1]

# Drop the original (redundant) column.
final_nominations = final_nominations.drop(["Additional Info"], axis=1)
final_nominations.head()


0                        Biutiful {'Uxbal
1             True Grit {'Rooster Cogburn
2    The Social Network {'Mark Zuckerberg
3      The King's Speech {'King George VI
4                127 Hours {'Aron Ralston
Name: Additional Info, dtype: object
0                        [Biutiful, Uxbal]
1             [True Grit, Rooster Cogburn]
2    [The Social Network, Mark Zuckerberg]
3      [The King's Speech, King George VI]
4                [127 Hours, Aron Ralston]
Name: Additional Info, dtype: object


,Year,Category,Nominee,Won,Movie,Character
0,2010,Actor -- Leading Role,Javier Bardem,0,Biutiful,Uxbal
1,2010,Actor -- Leading Role,Jeff Bridges,0,True Grit,Rooster Cogburn
2,2010,Actor -- Leading Role,Jesse Eisenberg,0,The Social Network,Mark Zuckerberg
3,2010,Actor -- Leading Role,Colin Firth,1,The King's Speech,King George VI
4,2010,Actor -- Leading Role,James Franco,0,127 Hours,Aron Ralston


# Writing the data to a SQL database

In [30]:
# Import SQLite, connect, and create a database by exporting the final_nominations dataframe to SQLite.
# The SQLite database will be named "nominations.db".
# The table containing the data from final_nominations will be named "nominations".

import sqlite3
conn = sqlite3.connect("nominations.db")

# Delete the table nominations if it already exists--in case you want to run this cell multiple times.
conn.execute("drop table if exists nominations;")

final_nominations.to_sql("nominations", conn, index=False, if_exists="append")

# Check to see that the columns in the table are correct.
c = conn.cursor()
info = c.execute("pragma table_info(nominations);").fetchall()
info

[(0, 'Year', 'INTEGER', 0, None, 0),
 (1, 'Category', 'TEXT', 0, None, 0),
 (2, 'Nominee', 'TEXT', 0, None, 0),
 (3, 'Won', 'INTEGER', 0, None, 0),
 (4, 'Movie', 'TEXT', 0, None, 0),
 (5, 'Character', 'TEXT', 0, None, 0)]

In [31]:
# Query the database to make sure the database creation worked.
head = c.execute("select * from nominations limit 10;").fetchall()
for row in head:
    print(row)

(2010, 'Actor -- Leading Role', 'Javier Bardem', 0, 'Biutiful', 'Uxbal')
(2010, 'Actor -- Leading Role', 'Jeff Bridges', 0, 'True Grit', 'Rooster Cogburn')
(2010, 'Actor -- Leading Role', 'Jesse Eisenberg', 0, 'The Social Network', 'Mark Zuckerberg')
(2010, 'Actor -- Leading Role', 'Colin Firth', 1, "The King's Speech", 'King George VI')
(2010, 'Actor -- Leading Role', 'James Franco', 0, '127 Hours', 'Aron Ralston')
(2010, 'Actor -- Supporting Role', 'Christian Bale', 1, 'The Fighter', 'Dicky Eklund')
(2010, 'Actor -- Supporting Role', 'John Hawkes', 0, "Winter's Bone", 'Teardrop')
(2010, 'Actor -- Supporting Role', 'Jeremy Renner', 0, 'The Town', 'James Coughlin')
(2010, 'Actor -- Supporting Role', 'Mark Ruffalo', 0, 'The Kids Are All Right', 'Paul')
(2010, 'Actor -- Supporting Role', 'Geoffrey Rush', 0, "The King's Speech", 'Lionel Logue')


# Creating relations in SQLite

We want to add more information to our dataset--namely, information about who hosted the ceremonies for the nominations in question. There would be a lot of repeated values if we entered this into a new column in the nominations table. We can be more efficient by creating a separate table for ceremonies, where each row represents a ceremony. We will create a new table for nominations, nominations_two. This table does not include the year, but instead includes ceremony_id, which refers to a row in ceremonies, giving us information about the year and host.

In [32]:
# This list of tuples gives the host for each year's ceremony from 2000 to 2010. Source: Wikipedia.
years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal"),
            ]

# Delete the table ceremonies if it already exists--in case you want to run this cell multiple times.
conn.execute("drop table if exists ceremonies;")

# Create a new table for ceremonies with the schema containing columns for the year and the host.
create_ceremonies = "create table ceremonies(id integer primary key, year integer, host text);"
conn.execute(create_ceremonies)

# Use executemany to enter all the years_hosts tuples into ceremonies
insert_query = "insert into ceremonies (year, host) values (?,?);"
conn.executemany(insert_query, years_hosts)

ceremonies = conn.execute("select * from ceremonies;").fetchall()
ceremonies

[(1, 2010, 'Steve Martin'),
 (2, 2009, 'Hugh Jackman'),
 (3, 2008, 'Jon Stewart'),
 (4, 2007, 'Ellen DeGeneres'),
 (5, 2006, 'Jon Stewart'),
 (6, 2005, 'Chris Rock'),
 (7, 2004, 'Billy Crystal'),
 (8, 2003, 'Steve Martin'),
 (9, 2002, 'Whoopi Goldberg'),
 (10, 2001, 'Steve Martin'),
 (11, 2000, 'Billy Crystal')]

In [33]:
ceremonies_info = conn.execute("pragma table_info(ceremonies)").fetchall()
ceremonies_info

[(0, 'id', 'integer', 0, None, 1),
 (1, 'year', 'integer', 0, None, 0),
 (2, 'host', 'text', 0, None, 0)]

In [34]:
# Turn on foreign key constraints to avoid inserting rows with foreign key values that don't exist.
conn.execute("pragma foreign_keys = on;")

In [35]:
# Delete the table nominations_two if it already exists--in case you want to run this cell multiple times.
conn.execute("drop table if exists nominations_two;")

# Create table "nominations_two" with the schema we want.
# We want all the info from nominations except the year, plus a foreign key for ceremony_id.
create_nominations_two = '''create table nominations_two 
(id integer primary key, 
category text, 
nominee text, 
movie text, 
character text, 
won text,
ceremony_id integer,
foreign key(ceremony_id) references ceremonies(id));
'''

conn.execute(create_nominations_two)


# Use an inner join to select all of the data needed from nominations and ceremonies to populate nominations_two.
nom_query = '''
select ceremonies.id as ceremony_id, nominations.category as category, 
nominations.nominee as nominee, nominations.movie as movie, 
nominations.character as character, nominations.won as won
from nominations
inner join ceremonies 
on nominations.year == ceremonies.year
;
'''

joined_nominations = conn.execute(nom_query).fetchall()


# Populate nominations_two using executemany. 
insert_nominations_two = "insert into nominations_two (ceremony_id, category, nominee, movie, character, won) values (?,?,?,?,?,?);"
conn.executemany(insert_nominations_two, joined_nominations)

first_five = conn.execute("select * from nominations_two limit 5;").fetchall()
for row in first_five:
    print(row)

(1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', '0', 1)
(2, 'Actor -- Leading Role', 'Jeff Bridges', 'True Grit', 'Rooster Cogburn', '0', 1)
(3, 'Actor -- Leading Role', 'Jesse Eisenberg', 'The Social Network', 'Mark Zuckerberg', '0', 1)
(4, 'Actor -- Leading Role', 'Colin Firth', "The King's Speech", 'King George VI', '1', 1)
(5, 'Actor -- Leading Role', 'James Franco', '127 Hours', 'Aron Ralston', '0', 1)


In [36]:
# In typical workflow, we would delete the original nominations table and rename the new table back to the original name.
# I am commenting this step out because I want to retain both tables in order to run the cells in this notebook repeatedly. 
# Moving forward, I will continue working with nominations_two.
'''
conn.execute("drop table nominations;")
conn.execute("alter table nominations_two rename to nominations;")
'''

'\nconn.execute("drop table nominations;")\nconn.execute("alter table nominations_two rename to nominations;")\n'

# Creating a join table

In this step, we will create three separate tables: a movies table, in which each row represents a movie; an actors table, in which each row represents an actor, and a join table, in which each row represents a nomination and contains the foreign keys that refer to the movie and actor pertaining to that nomination.

In [37]:
# Delete the three tables if they already exist--in case you want to run this cell multiple times.
conn.execute("drop table if exists movies;")
conn.execute("drop table if exists actors;")
conn.execute("drop table if exists movies_actors;")

# Create the three tables with the appropriate schemas.
conn.execute("create table movies (id integer primary key, movie text);")
conn.execute("create table actors (id integer primary key, actor text);")
conn.execute("create table movies_actors (id integer primary key, movie_id integer references movies(id), actor_id integer references actors(id));")


# Populate movies. 
insert_movies = "insert into movies (movie) select distinct movie from nominations_two;"
conn.execute(insert_movies)

five_movies = conn.execute("select * from movies limit 5;").fetchall()
for row in five_movies:
    print(row)

    
# Populate actors.
insert_actors = "insert into actors (actor) select distinct nominee from nominations_two;"
conn.execute(insert_actors)

five_actors = conn.execute("select * from actors limit 5;").fetchall()
for row in five_actors:
    print(row)
    
    
# Populate the join table.
pairs_query = "select movie, nominee from nominations_two;"
movie_actor_pairs = conn.execute(pairs_query).fetchall()

join_table_insert = "insert into movies_actors (movie_id, actor_id) values ((select id from movies where movie == ?), (select id from actors where actor == ?));"
conn.executemany(join_table_insert, movie_actor_pairs)

# Display the beginning of the join table.
twenty = conn.execute("select * from movies_actors limit 20;").fetchall()
twenty


(1, 'Biutiful')
(2, 'True Grit')
(3, 'The Social Network')
(4, "The King's Speech")
(5, '127 Hours')
(1, 'Javier Bardem')
(2, 'Jeff Bridges')
(3, 'Jesse Eisenberg')
(4, 'Colin Firth')
(5, 'James Franco')


[(1, 1, 1),
 (2, 2, 2),
 (3, 3, 3),
 (4, 4, 4),
 (5, 5, 5),
 (6, 6, 6),
 (7, 7, 7),
 (8, 8, 8),
 (9, 9, 9),
 (10, 4, 10),
 (11, 9, 11),
 (12, 10, 12),
 (13, 7, 13),
 (14, 11, 14),
 (15, 12, 15),
 (16, 6, 16),
 (17, 4, 17),
 (18, 6, 18),
 (19, 2, 19),
 (20, 13, 20)]

This concludes the project. We now have some nice, efficient tables that relate to each other and help us look at data about Academy Award actor/actress nominations from 2000 to 2010. 

In [38]:
# Close the connection. 
conn.close()